In [7]:
!dir

import os
# Root directory of the project
ROOT_DIR = os.path.abspath("G:\\My Drive\\FIRE COML\\2018\\Summer Fellowship\\Project\\MaskTrack_RCNN")

 Volume in drive C is Windows
 Volume Serial Number is D626-90F1

 Directory of C:\Users\rmdu\MaskTrack_RCNN

06/14/2018  11:49 AM    <DIR>          .
06/14/2018  11:49 AM    <DIR>          ..
06/14/2018  11:46 AM               627 .gitignore
06/14/2018  11:49 AM    <DIR>          .idea
06/14/2018  11:46 AM    <DIR>          assets
06/14/2018  11:46 AM    <DIR>          images
06/14/2018  11:46 AM             1,118 LICENSE
06/14/2018  11:46 AM                60 MANIFEST.in
06/14/2018  11:51 AM    <DIR>          mrcnn
06/14/2018  11:51 AM    <DIR>          project
06/14/2018  11:46 AM               108 README.md
06/14/2018  11:46 AM               130 requirements.txt
06/14/2018  11:46 AM    <DIR>          samples
06/14/2018  11:46 AM               102 setup.cfg
06/14/2018  11:46 AM             2,586 setup.py
               7 File(s)          4,731 bytes
               8 Dir(s)  95,993,012,224 bytes free


In [8]:
import os
import re
from mrcnn import config, utils
import skimage.io
import numpy as np
from os.path import join, isfile

import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from project import wad_data


# TRAIN IMAGES ISSUE
# No. Files in video lists = 42,369
# No. Files in train_color = 39,222
# No. Files in train_label = 37,689



## Configuration

In [9]:


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib qt5 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

## Dataset

load_mask has been tested. green light .. green light

In [13]:
from project import wad_data

# Training dataset
dataset_train = wad_data.WADDataset()

dataset_train.load_data("C:\Users\rmdu\MaskTrack_RCNN\dataset\wad", "train")

dataset_train.prepare()

Using pickle file for train-all: train-all_image_info.pkl


In [15]:
print("Image Count: {}".format(len(dataset_train.image_ids)))
#print("Class Count: {}".format(dataset_train.num_classes))

Image Count: 37689


In [18]:
import numpy as np
image_ids = np.random.choice(dataset_train.image_ids, 4)

#for image_id in image_ids:
image_id = image_ids[0]
image = dataset_train.load_image(image_id)
mask, class_ids = dataset_train.load_mask(image_id)
visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

## Testing

In [0]:
# Test dataset 

dataset_test = wad_data.WADDataset()

dataset_test.load_WAD("G:\\Team Drives\\COML-Summer-2018\\Data\\CVPR-WAD-2018", "train")

dataset_test.prepare()
